In [3]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
import os 
os.chdir('/content/drive/MyDrive/EEG Detection')
from functions import AddNoActivity, scaling
from average_metrics import average_bal_acc_score, average_f1_score, average_acc_score, average_hamming_loss
from classifiers import SupportVectorMachine, LogisticRegression, RandomForest, KNearestNeighbour, AdaBoost, XGBoost
from sklearn.model_selection import train_test_split
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
whole_data = []
for subject in range(1,11):
    for i in range(1,5):
        ts = pd.read_csv(f"/content/drive/MyDrive/train/train/subj{subject}_series{i}_data.csv")
        te = pd.read_csv(f"/content/drive/MyDrive/train/train/subj{subject}_series{i}_events.csv")
        temp = pd.concat([ts,te], axis = 1)        
        whole_data_temp = AddNoActivity(temp)
        whole_data.append(whole_data_temp)

In [4]:
data = pd.concat(whole_data)

In [5]:
data

,id,Fp1,Fp2,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,C3,Cz,C4,T8,TP9,CP5,CP1,CP2,CP6,TP10,P7,P3,Pz,P4,P8,PO9,O1,Oz,O2,PO10,id,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased,NoActivity
0,subj1_series1_0,-31,363,211,121,211,15,717,279,35,158,543,-166,192,230,573,860,128,59,272,473,325,379,536,348,383,105,607,289,459,173,120,704,subj1_series1_0,0,0,0,0,0,0,1
1,subj1_series1_1,-29,342,216,123,222,200,595,329,43,166,495,-138,201,233,554,846,185,47,269,455,307,368,529,327,369,78,613,248,409,141,83,737,subj1_series1_1,0,0,0,0,0,0,1
2,subj1_series1_2,-172,278,105,93,222,511,471,280,12,177,534,-163,198,207,542,768,145,52,250,452,273,273,511,319,355,66,606,320,440,141,62,677,subj1_series1_2,0,0,0,0,0,0,1
3,subj1_series1_3,-272,263,-52,99,208,511,428,261,27,180,525,-310,212,221,542,808,115,41,276,432,258,241,521,336,356,71,568,339,437,139,58,592,subj1_series1_3,0,0,0,0,0,0,1
4,subj1_series1_4,-265,213,-67,99,155,380,476,353,32,165,507,-320,242,230,545,865,180,89,288,444,275,275,550,324,346,76,547,343,446,171,67,581,subj1_series1_4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211220,subj10_series4_211220,-31,-172,39,-273,-300,-151,-209,222,-76,107,-120,-122,228,-233,-56,-151,27,88,-63,71,38,90,-147,-2,-76,43,351,-251,81,-89,-201,-249,subj10_series4_211220,0,0,0,0,0,0,1
211221,subj10_series4_211221,-47,-174,-15,-232,-297,-151,-193,152,-74,115,-123,-93,215,-235,-57,-148,60,34,-51,74,12,53,-125,-8,-85,26,322,-234,82,-106,-241,-280,subj10_series4_211221,0,0,0,0,0,0,1
211222,subj10_series4_211222,-78,-227,72,-240,-312,-166,-211,404,-45,91,-116,-256,266,-236,-78,-130,-40,104,-39,52,22,4,-77,7,-65,12,320,-202,81,-90,-251,-244,subj10_series4_211222,0,0,0,0,0,0,1
211223,subj10_series4_211223,-15,-177,185,-191,-286,-176,-198,431,-41,97,-68,-209,282,-238,-54,-114,-18,134,-11,57,35,106,-49,31,-52,27,336,-181,77,-65,-244,-242,subj10_series4_211223,0,0,0,0,0,0,1


In [6]:
s1 = data[data['NoActivity'] == 1]

In [7]:
s0 = data[data['NoActivity'] == 0]

In [8]:
s1 = s1.sample(n = (len(s0) + 00))

In [9]:
s1

,id,Fp1,Fp2,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,C3,Cz,C4,T8,TP9,CP5,CP1,CP2,CP6,TP10,P7,P3,Pz,P4,P8,PO9,O1,Oz,O2,PO10,id,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased,NoActivity
79498,subj7_series3_79498,220,341,542,130,-53,-73,-41,325,-63,-142,283,-89,285,-76,56,163,178,-97,-142,-189,-110,153,522,-89,17,-144,141,92,203,-123,-26,-61,subj7_series3_79498,0,0,0,0,0,0,1
63253,subj7_series1_63253,303,229,555,-606,-53,134,-264,-33,2,-112,-119,-422,-104,8,-228,-181,-22,-194,-86,17,-211,-89,-240,-158,135,52,-68,-498,119,-120,-346,-251,subj7_series1_63253,0,0,0,0,0,0,1
205760,subj5_series3_205760,212,487,402,12,305,433,400,278,133,3,155,3,239,-15,76,342,145,-140,4,7,35,24,-270,105,-45,315,195,-263,-128,-203,-246,-188,subj5_series3_205760,0,0,0,0,0,0,1
17477,subj10_series1_17477,-392,-190,-302,-528,-664,-292,-490,-55,-294,-384,-581,-446,-87,-587,-52,-433,-506,-755,-178,-116,-599,-298,-329,-429,189,-248,-79,-508,-203,-698,-464,-447,subj10_series1_17477,0,0,0,0,0,0,1
92185,subj7_series1_92185,334,223,251,-209,56,105,112,52,-41,61,529,-482,-79,-79,-74,18,-80,-110,-137,-141,-223,135,-15,-260,146,-171,39,-167,109,-115,-362,-166,subj7_series1_92185,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36147,subj6_series2_36147,95,87,520,-64,-34,188,22,180,496,-91,9,597,100,84,-123,-142,386,22,-195,130,-162,62,135,-38,170,204,223,-58,541,3,-64,-101,subj6_series2_36147,0,0,0,0,0,0,1
20212,subj7_series4_20212,-14,-33,497,136,-180,103,-50,244,-64,-210,-9,37,46,-179,39,110,17,-136,-148,-23,-182,-42,124,-207,40,-18,-54,-1,293,2,-16,-78,subj7_series4_20212,0,0,0,0,0,0,1
25299,subj5_series3_25299,-273,-235,755,326,112,341,-195,520,137,18,194,54,411,43,-72,63,464,184,214,237,12,720,250,574,327,383,124,124,373,173,49,-64,subj5_series3_25299,0,0,0,0,0,0,1
46252,subj2_series1_46252,178,-41,-617,-341,-73,-174,-158,-213,-376,-379,-30,-381,206,-71,-492,37,-518,-689,-237,-368,-299,210,-148,497,136,-305,123,-105,-74,-56,-397,-88,subj2_series1_46252,0,0,0,0,0,0,1


In [10]:
s = s1.append(s0)

In [11]:
s

,id,Fp1,Fp2,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,C3,Cz,C4,T8,TP9,CP5,CP1,CP2,CP6,TP10,P7,P3,Pz,P4,P8,PO9,O1,Oz,O2,PO10,id,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased,NoActivity
79498,subj7_series3_79498,220,341,542,130,-53,-73,-41,325,-63,-142,283,-89,285,-76,56,163,178,-97,-142,-189,-110,153,522,-89,17,-144,141,92,203,-123,-26,-61,subj7_series3_79498,0,0,0,0,0,0,1
63253,subj7_series1_63253,303,229,555,-606,-53,134,-264,-33,2,-112,-119,-422,-104,8,-228,-181,-22,-194,-86,17,-211,-89,-240,-158,135,52,-68,-498,119,-120,-346,-251,subj7_series1_63253,0,0,0,0,0,0,1
205760,subj5_series3_205760,212,487,402,12,305,433,400,278,133,3,155,3,239,-15,76,342,145,-140,4,7,35,24,-270,105,-45,315,195,-263,-128,-203,-246,-188,subj5_series3_205760,0,0,0,0,0,0,1
17477,subj10_series1_17477,-392,-190,-302,-528,-664,-292,-490,-55,-294,-384,-581,-446,-87,-587,-52,-433,-506,-755,-178,-116,-599,-298,-329,-429,189,-248,-79,-508,-203,-698,-464,-447,subj10_series1_17477,0,0,0,0,0,0,1
92185,subj7_series1_92185,334,223,251,-209,56,105,112,52,-41,61,529,-482,-79,-79,-74,18,-80,-110,-137,-141,-223,135,-15,-260,146,-171,39,-167,109,-115,-362,-166,subj7_series1_92185,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210024,subj10_series4_210024,-234,32,-40,-434,-272,-181,49,-154,-112,127,42,-204,-155,-389,38,-31,-141,-228,-214,-52,-97,120,-226,-418,-338,18,95,-233,-56,-403,-264,-147,subj10_series4_210024,0,0,0,0,0,1,0
210025,subj10_series4_210025,-317,111,-129,-426,-263,-178,87,-306,-122,104,49,-102,-154,-388,30,-43,-156,-180,-229,-50,-89,141,-170,-413,-342,24,119,-229,-34,-402,-262,-153,subj10_series4_210025,0,0,0,0,0,1,0
210026,subj10_series4_210026,-363,77,-187,-428,-273,-162,48,10,-115,99,30,-157,-139,-417,10,-61,-99,-251,-249,-69,-135,98,-216,-420,-338,-9,107,-204,-35,-408,-278,-225,subj10_series4_210026,0,0,0,0,0,1,0
210027,subj10_series4_210027,-299,71,-162,-432,-270,-139,75,133,-89,118,27,-175,-116,-407,20,-69,-130,-237,-233,-57,-130,126,-165,-406,-352,9,116,-166,-7,-351,-228,-180,subj10_series4_210027,0,0,0,0,0,1,0


In [12]:
s.reset_index(drop = True, inplace = True)

In [13]:
s.drop('id', axis = 1, inplace = True)

In [14]:
x = s.iloc[:,0:32]

In [15]:
y = s.iloc[:,32:]

In [16]:
x

,Fp1,Fp2,F7,F3,Fz,F4,F8,FC5,FC1,FC2,FC6,T7,C3,Cz,C4,T8,TP9,CP5,CP1,CP2,CP6,TP10,P7,P3,Pz,P4,P8,PO9,O1,Oz,O2,PO10
0,220,341,542,130,-53,-73,-41,325,-63,-142,283,-89,285,-76,56,163,178,-97,-142,-189,-110,153,522,-89,17,-144,141,92,203,-123,-26,-61
1,303,229,555,-606,-53,134,-264,-33,2,-112,-119,-422,-104,8,-228,-181,-22,-194,-86,17,-211,-89,-240,-158,135,52,-68,-498,119,-120,-346,-251
2,212,487,402,12,305,433,400,278,133,3,155,3,239,-15,76,342,145,-140,4,7,35,24,-270,105,-45,315,195,-263,-128,-203,-246,-188
3,-392,-190,-302,-528,-664,-292,-490,-55,-294,-384,-581,-446,-87,-587,-52,-433,-506,-755,-178,-116,-599,-298,-329,-429,189,-248,-79,-508,-203,-698,-464,-447
4,334,223,251,-209,56,105,112,52,-41,61,529,-482,-79,-79,-74,18,-80,-110,-137,-141,-223,135,-15,-260,146,-171,39,-167,109,-115,-362,-166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1896253,-234,32,-40,-434,-272,-181,49,-154,-112,127,42,-204,-155,-389,38,-31,-141,-228,-214,-52,-97,120,-226,-418,-338,18,95,-233,-56,-403,-264,-147
1896254,-317,111,-129,-426,-263,-178,87,-306,-122,104,49,-102,-154,-388,30,-43,-156,-180,-229,-50,-89,141,-170,-413,-342,24,119,-229,-34,-402,-262,-153
1896255,-363,77,-187,-428,-273,-162,48,10,-115,99,30,-157,-139,-417,10,-61,-99,-251,-249,-69,-135,98,-216,-420,-338,-9,107,-204,-35,-408,-278,-225
1896256,-299,71,-162,-432,-270,-139,75,133,-89,118,27,-175,-116,-407,20,-69,-130,-237,-233,-57,-130,126,-165,-406,-352,9,116,-166,-7,-351,-228,-180


In [17]:
y

,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased,NoActivity
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...
1896253,0,0,0,0,0,1,0
1896254,0,0,0,0,0,1,0
1896255,0,0,0,0,0,1,0
1896256,0,0,0,0,0,1,0


In [18]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3, random_state = 42)

In [19]:
x_train = scaling(x_train)

In [20]:
x_test = scaling(x_test)

In [21]:
XGBoost(x_train, x_test, y_train, y_test)

The average balanced accuracy score of the XGBoost classifier is  0.5214834499289764
The average accuracy score of the XGBoost classifier is  0.8632468523253543
The average F1 score of the XGBoost classifier is  0.09207988531148349
The average hamming loss of the XGBoost classifier is  0.13675314767464578


,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased,NoActivity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
568873,0.0,0.0,0.0,0.0,0.0,0.0,1.0
568874,0.0,0.0,0.0,0.0,0.0,0.0,0.0
568875,0.0,0.0,0.0,0.0,0.0,0.0,0.0
568876,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [22]:
AdaBoost(50, x_train, x_test, y_train, y_test)

The average balanced accuracy score of the AdaBoost classifier is  0.5184077489839594
The average accuracy score of the AdaBoost classifier is  0.8600468189765016
The average F1 score of the AdaBoost classifier is  0.0910845931470701
The average hamming loss of the AdaBoost classifier is  0.13995318102349838


,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased,NoActivity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
568873,0.0,0.0,0.0,0.0,0.0,0.0,1.0
568874,0.0,0.0,0.0,0.0,0.0,0.0,1.0
568875,0.0,0.0,0.0,0.0,0.0,0.0,1.0
568876,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [23]:
RandomForest(100, x_train, x_test, y_train, y_test)

The average balanced accuracy score of the random forest classifier is  0.8078625686814815
The average accuracy score of the random forest classifier is  0.9510922000348556
The average F1 score of the random forest classifier is  0.7522676354120371
The average hamming loss of the random forest classifier is  0.048907799965144425


,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased,NoActivity
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
568873,0.0,1.0,1.0,0.0,0.0,0.0,0.0
568874,0.0,0.0,0.0,0.0,0.0,0.0,0.0
568875,0.0,0.0,0.0,0.0,0.0,0.0,1.0
568876,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
KNearestNeighbour(5, x_train, x_test, y_train, y_test)

In [ ]:
LogisticRegression(x_train, x_test, y_train, y_test)